# Valdidation of Gram-Schmidt derivative expressions


Gram-Schmidt orthogonalization is a well-known technique where a non-orthogonal (yet independent) set of column vectors ${\bf q}_i\in\mathbb{R}^D$ is made orthogonal via

\begin{align}
 {\bf w}_i = {\bf q}_i - \sum_{j=1}^{i-1}\left(\frac{{\bf w}_j^T{\bf q}_i}{{\bf w}_j^T{\bf w}_j}\right){\bf w}_j, \quad i = 1,\cdots, d.
 \label{eq:GS}
\end{align}

That is, we start with ${\bf w}_1 := {\bf q}_1$, and for all subsequent vectors ${\bf q}_i$ we subtract the projections of ${\bf q}_i$ onto each vector ${\bf w}_j$ which has previously been orthogonalized. This leaves us with a orthogonal basis $ \left[{\bf w}_1({\bf q}_1)\;\;{\bf w}_2({\bf q}_1, {\bf q}_2)\;\;\cdots\;\;{\bf w}_d({\bf q}_1,{\bf q}_2\,\cdots,{\bf q}_d)\right]$. Finally, to obtain an orthonormal basis, each column vector can be divided by its length:

\begin{align}
\frac{{\bf w}_1({\bf q}_1)}{\lVert{\bf w}_1({\bf q}_1)\rVert_2},\;\;\frac{{\bf w}_2({\bf q}_1, {\bf q}_2)}{\lVert{\bf w}_2({\bf q}_1, {\bf q}_2)\rVert_2},\;\;\cdots\;\;,\frac{{\bf w}_d({\bf q}_1,{\bf q}_2\,\cdots,{\bf q}_d)}{\lVert{\bf w}_d({\bf q}_1,{\bf q}_2\,\cdots,{\bf q}_d)\rVert_2}
\end{align}

We have used matrix calculus to find a simple recurrence relation for the *derivatives* of the Gram-Schmidt vectors, starting with the gradient of the unnormalized vector:

\begin{align}
\boxed{
\frac{\partial{\bf w}_1}{\partial{\bf q}_1} =: D_{11} = I_D \\ 
\frac{\partial{\bf w}_i}{\partial{\bf q}_i} =: D_{ii} = D_{i-1,\, i-1} - \frac{{\bf w}_{i-1}{\bf w}_{i-1}^T}{{\bf w}_{i-1}^T{\bf w}_{i-1}},\quad i > 1, \\
%
\frac{\partial{\bf w}_i}{\partial{\bf q}_k} = \sum_{j=1}^{i-1}D_{ij}\frac{\partial{\bf w}_j}{\partial{\bf q}_k},\quad i\neq k, \quad  i > k, \\
%
D_{ij}:= 
%
-\frac{\partial}{\partial{\bf w}_j}\left[\left(\frac{{\bf w}_j^T{\bf q}_i}{{\bf w}_j^T{\bf w}_j}\right){\bf w}_j\right] =
%
-\left[\frac{1}{{\bf w}_j^T{\bf w}_j}\;{\bf w}_j{\bf q}_i^T - \frac{2{\bf w}_j^T{\bf q}_i}{({\bf w}_j^T{\bf w}_j)^2}\;{\bf w}_j{\bf w}_j^T + \frac{{\bf w}_j^T{\bf q}_i}{{\bf w}_j^T{\bf w}_j}\; I_D\right],\quad i\neq j,\quad i > j.}
%
\end{align}


Here, $I_D$ is the $D$-dimensional identity matrix. Note that we have separate expressions for the "normal" derivatives $\partial{\bf w}_i/\partial{\bf q}_i$ and "shear" derivatives $\partial{\bf w}_i/\partial{\bf q}_j$ (where $i\neq j$), which we will compute in a recursive manner, starting with $i=1$.

It is useful to note that due to ${\bf w}_1 = {\bf w}_1({\bf q}_1),\;\;{\bf w}_2 = {\bf w}_2({\bf q}_1, {\bf q}_2),\;\;\cdots\;\;,{\bf w}_d = {\bf w}_d({\bf q}_1,{\bf q}_2\,\cdots,{\bf q}_d)$, we already know that $\partial{\bf w}_i/\partial{\bf q}_k=0$ if $i < k$.

### Goal
We will use this notebook to;

* validate the recurrence relations of Gram-Schmidt derivatives, using brute-force symbolic calulations of the gradients performed with Sympy, and 

* to verify our Python implementation of the recurrence relations (`gram_schmidt_dericatives.py`).

The derivation of the recurrence relations can be found ![here](./derivation.pdf). You need to have Numpy and Sympy installed. If not, uncomment the corresponding lines below.

# Validation of mathematical equations

In [35]:
#!pip install sympy
#!pip install numpy

In [36]:
import sympy as sp
import numpy as np

Here, we are creating $d=3$ independent symbolic basis vectors with $D=4$ entries each:

In [37]:
D = 4
d = 3
for i in range(D):
    for j in range(d):
        vars()['q_%d%d' % (i+1, j+1)] = sp.var('q_%d%d' % (i+1, j+1), real=True)

q1 = sp.Matrix([q_11, q_21, q_31, q_41])
q2 = sp.Matrix([q_12, q_22, q_32, q_42])
q3 = sp.Matrix([q_13, q_23, q_33, q_43])
q1

Matrix([
[q_11],
[q_21],
[q_31],
[q_41]])

In [38]:
# Gram-Schmidt orthogonalization. Need to add [0] after the inner product for division to work.
w1 = q1
w2 = q2 - (w1.T * q2)[0] / (w1.T * w1)[0] * w1 
w3 = q3 - (w1.T * q3)[0] / (w1.T * w1)[0] * w1 - (w2.T * q3)[0] / (w2.T * w2)[0] * w2 

A small sanity check to see if the ${\bf w}_i$ are orthogonal:

In [39]:
sp.simplify((w1.T * w2)[0])

0

Note that the ${\bf w}_i$ vectors quickly become a complicated expression of a very large number of $q_{ij}$ terms:

In [40]:
w3

Matrix([
[-q_11*(q_11*q_13 + q_21*q_23 + q_31*q_33 + q_41*q_43)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + q_13 - (-q_11*(q_11*q_12 + q_21*q_22 + q_31*q_32 + q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + q_12)*(q_13*(-q_11*(q_11*q_12 + q_21*q_22 + q_31*q_32 + q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + q_12) + q_23*(-q_21*(q_11*q_12 + q_21*q_22 + q_31*q_32 + q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + q_22) + q_33*(-q_31*(q_11*q_12 + q_21*q_22 + q_31*q_32 + q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + q_32) + q_43*(-q_41*(q_11*q_12 + q_21*q_22 + q_31*q_32 + q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + q_42))/((-q_11*(q_11*q_12 + q_21*q_22 + q_31*q_32 + q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + q_12)**2 + (-q_21*(q_11*q_12 + q_21*q_22 + q_31*q_32 + q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + q_22)**2 + (-q_31*(q_11*q_12 + q_21*q_22 + q_31*q_32 + q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + q_32)**2 + (-q_41*(q_11*q_12 +

Although the expression above is algebraic, it might seem to complicated to find the derivative of the ${\bf w}_i$ vectors with respect to the ${\bf q}_k$ vectors, hence the need for the recurrence relations. 

We now create a subroutine for computing the $D\times D$ matrix $D_{ij}$:


In [41]:
def D_ij(wj, qi):
    """
    Compute the D_ij matrices which appear in the gradients of the
    Gram-Schmidt vectors w_i wrt the original unnormalized vectors q_k.
    The relation below is only defined for i unequal to j:

    D_ij = 1/(w_j^Tw_j) [w_jq_i^T - (2w_j^Tq_i)/(w_j^Tw_j)*w_jw_j^T + w_j^Tq_i*I]
    where I is the D x D identity matrix.

    Parameters
    ----------
    w_j : sympy array, shape (D,1)
        The orthogonal, but not yet normalized, vector w_i, i=1,...,d.
    q_i : sympy array, shape (D,1)
        An orginal, non-orthonogal, vector q_j, j=1,...,d.

    Returns
    -------
    D_ij : array
        The D x D matrix described above.

    """
    
    term1 = wj * qi.T
    term2 = 2 * (wj.T * qi)[0] / (wj.T * wj)[0] * wj * wj.T
    term3 = (wj.T * qi)[0] * sp.eye(D)
    
    return -(term1 - term2 + term3) / (wj.T * wj)[0]

##  Verification of $\partial{\bf w}_2/\partial{\bf q}_1$

First we compute the "shear" derivative $\partial{\bf w}_2/\partial{\bf q}_1$:

\begin{align}
\frac{\partial{\bf w}_2}{\partial{\bf q}_1} = \sum_{j=1}^1 D_{2j}\frac{\partial{\bf w}_j}{\partial{\bf q}_1} = D_{21}\frac{\partial{\bf w}_1}{\partial{\bf q}_1} = D_{21}D_{11} = D_{21}
\end{align}

Below we'll compute $D_{21}$ and the symbolic derivative of $\partial{\bf w}_2/\partial{\bf q}_1$

In [42]:
# compute D_21
D_21 = D_ij(w1, q2)
dw2_dq1 = D_21
dw2_dq1

Matrix([
[(q_11**2*(2*q_11*q_12 + 2*q_21*q_22 + 2*q_31*q_32 + 2*q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) - 2*q_11*q_12 - q_21*q_22 - q_31*q_32 - q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2),                                      (q_11*q_21*(2*q_11*q_12 + 2*q_21*q_22 + 2*q_31*q_32 + 2*q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) - q_11*q_22)/(q_11**2 + q_21**2 + q_31**2 + q_41**2),                                      (q_11*q_31*(2*q_11*q_12 + 2*q_21*q_22 + 2*q_31*q_32 + 2*q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) - q_11*q_32)/(q_11**2 + q_21**2 + q_31**2 + q_41**2),                                      (q_11*q_41*(2*q_11*q_12 + 2*q_21*q_22 + 2*q_31*q_32 + 2*q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) - q_11*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2)],
[                                    (q_11*q_21*(2*q_11*q_12 + 2*q_21*q_22 + 2*q_31*q_32 + 2*q_41*q_42)/(q_11**2 + q_21**2 + q_31**2 + q_41**2) - q_12*q_21)/(q_11**2 + q_21**2 + q_31**2 + q_41**2), 

In [43]:
def sympy_derivative(wi, qk):
    """
    Use sympy to compute dw_i / dq_k

    Parameters
    ----------
    w_j : sympy array, shape (D,1)
        The orthogonal, but not yet normalized, vector w_i, i=1,...,d.
    q_i : sympy array, shape (D,1)
        An orginal, non-orthonogal, vector q_j, j=1,...,d.

    Returns
    -------
    dwi_dqk : sympy Matrix
        The D x D matrix gradient matrix.
    """
    dwi_dqk = sp.diff(wi, qk)
    # reshape to (D, D) and convert to Matrix type
    dwi_dqk = sp.Matrix(dwi_dqk[:, 0, :, 0]).T
    return dwi_dqk

Now we'll perform an elementwise comparison of our answer for $\partial{\bf w}_2/\partial{\bf q}_1$, and the sympy expression:

In [44]:
dw2_dq1_sympy = sympy_derivative(w2, q1)
sp.simplify(dw2_dq1 - dw2_dq1_sympy)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

This should yield all zeros.

## Verification of $\partial{\bf w}_2/\partial{\bf q}_2$

We now compute the "normal" derivative $\partial{\bf w}_2/\partial{\bf q}_2$:

\begin{align}
\frac{\partial{\bf w}_2}{\partial{\bf q}_2} 
= D_{11} - \frac{{\bf w}_1{\bf w}_1^T}{{\bf w}_1^T{\bf w}_1} =: D_{22}
\end{align}

In [45]:
# D_11 is the identity matrix by definition
D_11 = sp.eye(D)
dw1_dq1 = D_11
# compute D_22
dw2_dq2 = D_11 - w1 * w1.T / (w1.T * w1)[0]
D_22 = dw2_dq2
dw2_dq2

Matrix([
[-q_11**2/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + 1,   -q_11*q_21/(q_11**2 + q_21**2 + q_31**2 + q_41**2),   -q_11*q_31/(q_11**2 + q_21**2 + q_31**2 + q_41**2),   -q_11*q_41/(q_11**2 + q_21**2 + q_31**2 + q_41**2)],
[  -q_11*q_21/(q_11**2 + q_21**2 + q_31**2 + q_41**2), -q_21**2/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + 1,   -q_21*q_31/(q_11**2 + q_21**2 + q_31**2 + q_41**2),   -q_21*q_41/(q_11**2 + q_21**2 + q_31**2 + q_41**2)],
[  -q_11*q_31/(q_11**2 + q_21**2 + q_31**2 + q_41**2),   -q_21*q_31/(q_11**2 + q_21**2 + q_31**2 + q_41**2), -q_31**2/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + 1,   -q_31*q_41/(q_11**2 + q_21**2 + q_31**2 + q_41**2)],
[  -q_11*q_41/(q_11**2 + q_21**2 + q_31**2 + q_41**2),   -q_21*q_41/(q_11**2 + q_21**2 + q_31**2 + q_41**2),   -q_31*q_41/(q_11**2 + q_21**2 + q_31**2 + q_41**2), -q_41**2/(q_11**2 + q_21**2 + q_31**2 + q_41**2) + 1]])

Now we calculate the sympy expression of $\partial{\bf w}_2/\partial{\bf q}_2$ and the difference:

In [46]:
# use sympy to compute dw_2 / dq_1
dw2_dq2_sympy = sympy_derivative(w2, q2)
sp.simplify(dw2_dq2 - dw2_dq2_sympy)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

## Verification of $\partial{\bf w}_3/\partial{\bf q}_1$

The analysis below will continue in a similar vein as performed above.

\begin{align}
\frac{\partial{\bf w}_3}{\partial{\bf q}_1} = 
\sum_{j=1}^2 D_{3j}\frac{\partial{\bf w}_j}{\partial{\bf q}_1} = 
%
D_{31}\frac{\partial{\bf w}_1}{\partial{\bf q}_1} + D_{32}\frac{\partial{\bf w}_2}{\partial{\bf q}_1} = D_{31}D_{11} + D_{32}D_{21}.
\end{align}

In [47]:
# compute D_31
D_31 = D_ij(w1, q3)
# compute D_32
D_32 = D_ij(w2, q3)
# compute dw3_dq1
dw3_dq1 = D_31 + D_32 * dw2_dq1
# compute Sympy answer
dw3_dq1_sympy = sympy_derivative(w3, q1)

However, at this point the symbolic expression are so large that simplifying the expressions using `sp.simplify(dw3_dq1 - dw3_dq1_sympy)` takes a long time to compute. To speed things up we will write the following subroutine which evaluates both symbolic expressions at randomly drawn values for the $q_{ij}$ variables.

In [48]:
def eval_num(deriv1, deriv2):
    """
    Replaces all q_ij variables in two derivative matrices with random values.

    Parameters
    ----------
    deriv1 : sympy matrix, shape (D,D)
        A derivative matrix dwi_dqk.
    deriv2 : sympy matrix, shape (D,D)
        A derivative matrix dwi_dqk, should have exactly the same q_ij variables as deriv1.

    Returns
    -------
    num_deriv1, num_deriv2 : sympy Matrices
        The D x D matrix gradient matrices with all numerical values.
    """
    # store a copy of deriv1 and deriv2 to fill with random values
    num_deriv1 = deriv1
    num_deriv2 = deriv2

    # loop over all q_ij variables
    for q_ij in num_deriv1.free_symbols:
        # choose the same random value for both q_ij variables
        xi = np.random.rand()
        num_deriv1 = num_deriv1.subs(q_ij, xi)
        num_deriv2 = num_deriv2.subs(q_ij, xi)

    return num_deriv1, num_deriv2

In [49]:
num1, num2 = eval_num(dw3_dq1, dw3_dq1_sympy)
num1 - num2

Matrix([
[ 5.55111512312578e-16,   1.2490009027033e-16,  2.77555756156289e-16,  2.77555756156289e-17],
[-5.55111512312578e-17,  6.66133814775094e-16,  1.11022302462516e-16, -3.33066907387547e-16],
[-1.66533453693773e-16, -2.77555756156289e-17,   4.9960036108132e-16,  1.11022302462516e-16],
[ 1.66533453693773e-16,                     0, -5.55111512312578e-17,  3.88578058618805e-16]])

This should yield all zeros down to machine precision.

## Verification of $\partial{\bf w}_3/\partial{\bf q}_2$

\begin{align}
\frac{\partial{\bf w}_3}{\partial{\bf q}_2} = 
\sum_{j=1}^2 D_{3j}\frac{\partial{\bf w}_j}{\partial{\bf q}_2} = 
%
D_{31}\underbrace{\frac{\partial{\bf w}_1}{\partial{\bf q}_2}}_{0} + D_{32}\frac{\partial{\bf w}_2}{\partial{\bf q}_2} = D_{32}D_{22}.
\end{align}

All required $D_{ij}$ matrices have already been computed.

In [50]:
# compute dw3_dq2
dw3_dq2 = D_32 * dw2_dq2
# compute Sympy answer
dw3_dq2_sympy = sympy_derivative(w3, q2)
# evaluate difference numerically
num1, num2 = eval_num(dw3_dq2, dw3_dq2_sympy)
num1 - num2

Matrix([
[ 3.60822483003176e-16,  5.55111512312578e-17, 4.16333634234434e-17,                    0],
[-6.93889390390723e-18,                     0,                    0, 2.77555756156289e-17],
[-6.93889390390723e-18, -2.77555756156289e-17,                    0, 2.77555756156289e-17],
[-1.38777878078145e-17,  2.77555756156289e-17, 2.77555756156289e-17,                    0]])

## Verification of $\partial{\bf w}_3/\partial{\bf q}_3$

\begin{align}
\frac{\partial{\bf w}_3}{\partial{\bf q}_3} 
= D_{22} - \frac{{\bf w}_2{\bf w}_2^T}{{\bf w}_2^T{\bf w}_2} =: D_{33}
\end{align}

In [51]:
# compute dw3_dq3
dw3_dq3 = D_22 - w2 * w2.T / (w2.T * w2)[0]
# compute Sympy answer
dw3_dq3_sympy = sympy_derivative(w3, q3)
# compute difference symbolically, as the normal derivatives are not very complicated expressions
sp.simplify(dw3_dq3 - dw3_dq3_sympy)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

## Verification of gradients of normalized vectors ${\bf w}_i/\lVert{\bf w}_i\rVert_2$

Gram-Schmidt vectors are often normalized in practise, to obtain the set

\begin{align}
W = \bigg\{\frac{{\bf w}_1({\bf q}_1)}{\lVert{\bf w}_1({\bf q}_1)\rVert_2},\;\;\frac{{\bf w}_2({\bf q}_1, {\bf q}_2)}{\lVert{\bf w}_2({\bf q}_1, {\bf q}_2)\rVert_2},\;\;\cdots\;\;,\frac{{\bf w}_d({\bf q}_1,{\bf q}_2\,\cdots,{\bf q}_d)}{\lVert{\bf w}_d({\bf q}_1,{\bf q}_2\,\cdots,{\bf q}_d)\rVert_2}\bigg\}.
\end{align}

To find the derivatives of these vectors we can just premultiply $\partial{\bf w}_i/\partial{\bf q}_k$ with a matrix which only depends upon ${\bf w}_i$, to obtain the gradient of the corresponding normed vector:

\begin{align}
\boxed{\frac{\partial}{\partial{\bf q}_k}\left(\frac{{\bf w}_i}{\lVert{\bf w}_i\rVert_2}\right) = \left[\frac{I_D}{\lVert{\bf w}_i\rVert_2} - \frac{{\bf w}_i{\bf w}_i^T}{\lVert{\bf w}_i\rVert^3_2}\right]\frac{\partial{\bf w}_i}{\partial{\bf q}_k}.}
\end{align}

In [52]:
def normalize(w):
    """
    Subroutine to normalize a symbolic vector w. Also it computes the 
    normalizing matrix for derivatives of w / ||w||_2

    Parameters
    ----------
    w : sympy array, shape (D,)
        The array to be normalized

    Returns
    -------
    w_normed : sympy array, shape (D, )
        The normalized array w / ||w||_2
    
    w_deriv_norm_mat : sympy matrix, shape (D, D)
        The matrix used for computing the derivatives of w / ||w||_2
    """
    norm_w = 0.
    # loop ober all entries in w
    for w_i in w:
        # compute norm
        norm_w += w_i ** 2
    # return w / ||w||_2
    norm_w = sp.sqrt(norm_w)
    w_normed = w / norm_w

    # also compute the normalizing matrix for derivatives of w / ||w||_2
    w_deriv_norm_mat = sp.eye(D) / norm_w - w * w.T / norm_w ** 3

    return w_normed, w_deriv_norm_mat

Here we'll compute each ${\bf w}_i/\lVert{\bf w}_i\rVert_2$, where $\lVert{\bf w}_i\rVert_2 = \sqrt{w_{1i}^2 + w_{2i}^2 + \cdots + w_{Di}^2}$:

In [53]:
# compute the normed Gram-Schmidt vectors
w1_normed, w1_deriv_norm_mat = normalize(w1)
w2_normed, w2_deriv_norm_mat = normalize(w2)
w3_normed, w3_deriv_norm_mat = normalize(w3)
w1_normed

Matrix([
[q_11/sqrt(q_11**2 + q_21**2 + q_31**2 + q_41**2)],
[q_21/sqrt(q_11**2 + q_21**2 + q_31**2 + q_41**2)],
[q_31/sqrt(q_11**2 + q_21**2 + q_31**2 + q_41**2)],
[q_41/sqrt(q_11**2 + q_21**2 + q_31**2 + q_41**2)]])

Here we compute all reference derivatives $\partial\left({\bf w}_i/\lVert{\bf w}_i\rVert_2\right)/\partial{\bf q}_k$ in one shot:

In [54]:
# d(w1/||w1||_2) / dq1
dw1_dq1_normed_sympy = sympy_derivative(w1_normed, q1)
# d(w2/||w2||_2) / dq1
dw2_dq1_normed_sympy = sympy_derivative(w2_normed, q1)
# d(w2/||w2||_2) / dq2
dw2_dq2_normed_sympy = sympy_derivative(w2_normed, q2)
# d(w3/||w3||_2) / dq1
dw3_dq1_normed_sympy = sympy_derivative(w3_normed, q1)
# d(w3/||w3||_2) / dq2
dw3_dq2_normed_sympy = sympy_derivative(w3_normed, q2)
# d(w3/||w3||_2) / dq3
dw3_dq3_normed_sympy = sympy_derivative(w3_normed, q3)

Here we do the same using our expression for $\partial\left({\bf w}_i/\lVert{\bf w}_i\rVert_2\right)/\partial{\bf q}_k$

In [55]:
# d(w1/||w1||_2) / dq1
dw1_dq1_normed = w1_deriv_norm_mat * dw1_dq1
# d(w2/||w2||_2) / dq1
dw2_dq1_normed = w2_deriv_norm_mat * dw2_dq1
# d(w2/||w2||_2) / dq2
dw2_dq2_normed = w2_deriv_norm_mat * dw2_dq2
# d(w3/||w3||_2) / dq1
dw3_dq1_normed = w3_deriv_norm_mat * dw3_dq1
# d(w3/||w3||_2) / dq2
dw3_dq2_normed = w3_deriv_norm_mat * dw3_dq2
# d(w3/||w3||_2) / dq3
dw3_dq3_normed = w3_deriv_norm_mat * dw3_dq3

For each derivative we will again compute the difference with the Sympy reference value, using numerical evaluation where necessary:

* $\partial\left({\bf w}_1/\lVert{\bf w}_1\rVert_2\right)/\partial{\bf q}_1$ check

In [56]:
sp.simplify(dw1_dq1_normed - dw1_dq1_normed_sympy)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

* $\partial\left({\bf w}_2/\lVert{\bf w}_2\rVert_2\right)/\partial{\bf q}_1$ check

In [57]:
sp.simplify(dw2_dq1_normed - dw2_dq1_normed_sympy)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

* $\partial\left({\bf w}_2/\lVert{\bf w}_2\rVert_2\right)/\partial{\bf q}_2$ check

In [58]:
sp.simplify(dw2_dq2_normed - dw2_dq2_normed_sympy)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

* $\partial\left({\bf w}_3/\lVert{\bf w}_3\rVert_2\right)/\partial{\bf q}_1$ check

In [59]:
num1, num2 = eval_num(dw3_dq1_normed, dw3_dq1_normed_sympy)
num1 - num2

Matrix([
[-4.44089209850063e-15,  3.10862446895044e-15,  8.88178419700125e-16,  1.66533453693773e-15],
[ 1.77635683940025e-15,  -1.4432899320127e-15, -2.22044604925031e-15,  6.66133814775094e-16],
[ 3.60822483003176e-16,  1.55431223447522e-15,  -5.6621374255883e-15,  4.21884749357559e-15],
[-9.43689570931383e-16, -3.33066907387547e-16,  1.38777878078145e-15, -2.22044604925031e-16]])

* $\partial\left({\bf w}_3/\lVert{\bf w}_3\rVert_2\right)/\partial{\bf q}_2$ check

In [60]:
num1, num2 = eval_num(dw3_dq2_normed, dw3_dq2_normed_sympy)
num1 - num2

Matrix([
[-5.55111512312578e-17,  1.11022302462516e-16, -1.11022302462516e-16,  8.32667268468867e-17],
[ 3.33066907387547e-16, -3.33066907387547e-16,  1.11022302462516e-16,  3.33066907387547e-16],
[-4.44089209850063e-16,  2.22044604925031e-16, -3.33066907387547e-16,  3.33066907387547e-16],
[ 1.11022302462516e-16, -1.11022302462516e-16,  6.66133814775094e-16, -1.11022302462516e-16]])

* $\partial\left({\bf w}_3/\lVert{\bf w}_3\rVert_2\right)/\partial{\bf q}_3$ check

In [61]:
num1, num2 = eval_num(dw3_dq3_normed, dw3_dq3_normed_sympy)
num1 - num2

Matrix([
[-8.32667268468867e-17, -2.77555756156289e-16, 2.22044604925031e-16,  1.11022302462516e-16],
[-5.55111512312578e-17, -5.55111512312578e-17, 2.22044604925031e-16,                     0],
[-2.22044604925031e-16,  2.77555756156289e-16, 4.44089209850063e-16, -4.44089209850063e-16],
[ 1.11022302462516e-16,  5.55111512312578e-17,                    0,  4.44089209850063e-16]])

# Verification of Python implementation

The results above show the mathematical validity of the recurrence relations for the Gram-Schmidt derivatives. The next step is to verify if we implemented these equations correctly in Python. This implementation can be found in `gram_schmidt_derivatives.py`, which we import below.

In [62]:
import gram_schmidt_derivatives as gsd

Here we randomly generate some ${\bf q}_i$ vectors and pass these to the Python subroutine.

In [63]:
Q = np.random.rand(D, d)
gs_deriv = gsd.Gram_Schmidt_Derivatives(Q)

Computing Gram-Schmidt orthogonalization...
done.


Next we retrieve the Python derivatives, which are stored in a dict with `(i, k)` tuples as indices, corresponding to $\partial\left({\bf w}_i\right)/\partial{\bf q}_k$

In [64]:
dw_dq_python = gs_deriv.get_w_derivatives()
dw_dq_python[(2,1)]

array([[-0.44052517,  0.19043534, -0.08075742, -0.0377278 ],
       [ 0.19817804,  0.05847759, -0.23990129, -0.11207575],
       [ 0.04155361,  0.11861807, -0.55753945, -0.02349984],
       [ 0.13242033,  0.37800438, -0.16029934, -0.58212514]])

We will evaluate these derivatives against the validated recurrence relations. To do so, we must numerically evaluate the latter for the same random $q_{ij}$ values stored in `Q`. The following subroutine does this:

In [65]:
def eval_num_at_Q(dw_dq, Q):
    """
    Evaluate the symbolic derivative dw_dq at the numerical values of Q.

    Parameters
    ----------
    dw_dq : sympy Matrix, size (D, D)
        Gram-Schmidt derivative matrix.
    Q : array, size (D, d)
        Numerical values for the q_ij variables.

    Returns
    -------
    array, size (D,D)
        dw_dq evaluated at Q.

    """
    # make a copy of dw_dq in which we can substitute numerical values
    num_deriv = dw_dq
    # loop over all q_ij variables of dw_dq
    for q_ij in dw_dq.free_symbols:
        # get the name of each variable, e.g. 'q_12'
        var_name = q_ij.name
        # extract the indices i, j of q_ij
        i = int(var_name[2]) - 1
        j = int(var_name[3]) - 1
        # substitue q_ij with the value Q[i, j] in num_deriv
        num_deriv = num_deriv.subs(q_ij, Q[i, j])
    # return a numpy array
    return np.array(num_deriv).astype(np.float64)

Below we numerically evaluate the difference between the Python implementation and the Sympy recurrence relations for $\partial{\bf w}_i/\partial{\bf q}_k$:

In [66]:
# evaluate python implementation of dw_2 / dq_1
dw_dq_ref = eval_num_at_Q(dw2_dq1, Q)
print(dw_dq_ref - dw_dq_python[(2,1)])

# evaluate python implementation of dw_2 / dq_2
dw_dq_ref = eval_num_at_Q(dw2_dq2, Q)
print(dw_dq_ref - dw_dq_python[(2,2)])

# evaluate python implementation of dw_3 / dq_1
dw_dq_ref = eval_num_at_Q(dw3_dq1, Q)
print(dw_dq_ref - dw_dq_python[(3,1)])

# evaluate python implementation of dw_3 / dq_2
dw_dq_ref = eval_num_at_Q(dw3_dq2, Q)
print(dw_dq_ref - dw_dq_python[(3,2)])

# evaluate python implementation of dw_3 / dq_3
dw_dq_ref = eval_num_at_Q(dw3_dq3, Q)
print(dw_dq_ref - dw_dq_python[(3,3)])

[[-5.55111512e-17 -2.77555756e-17 -2.77555756e-17 -1.38777878e-17]
 [-5.55111512e-17  1.38777878e-17 -2.77555756e-17 -4.16333634e-17]
 [-1.38777878e-17  0.00000000e+00  0.00000000e+00 -1.04083409e-17]
 [-2.77555756e-17 -5.55111512e-17 -5.55111512e-17  0.00000000e+00]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.77555756e-17]
 [ 0.00000000e+00 -1.11022302e-16  0.00000000e+00 -5.55111512e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.77555756e-17 -5.55111512e-17  0.00000000e+00 -1.11022302e-16]]
[[-2.22044605e-16  8.32667268e-17 -5.20417043e-17 -3.46944695e-18]
 [-1.11022302e-16  2.49800181e-16 -8.32667268e-17  2.77555756e-17]
 [ 8.32667268e-17 -1.38777878e-17  1.66533454e-16  1.11022302e-16]
 [ 2.77555756e-17  2.77555756e-17  5.55111512e-17  3.33066907e-16]]
[[ 3.33066907e-16 -1.38777878e-16  4.85722573e-17 -2.77555756e-17]
 [ 0.00000000e+00 -5.55111512e-17  1.94289029e-16  6.93889390e-17]
 [ 0.00000000e+00  1.80411242e-16  2.22044605e-16 -2.220446

Again, these should all yield a matrix containing only zeros to machine precision. Finally, we do the same for all $\partial\left({\bf w}_i/\lVert{\bf w}_i\rVert_2\right)/\partial{\bf q}_k$

In [67]:
dw_dq_python_normed = gs_deriv.get_normed_w_derivatives()

In [68]:
# evaluate python implementation of dw_2 / dq_1
dw_dq_ref = eval_num_at_Q(dw2_dq1_normed, Q)
print(dw_dq_ref - dw_dq_python_normed[(2,1)])

# evaluate python implementation of dw_2 / dq_2
dw_dq_ref = eval_num_at_Q(dw2_dq2_normed, Q)
print(dw_dq_ref - dw_dq_python_normed[(2,2)])

# evaluate python implementation of dw_3 / dq_1
dw_dq_ref = eval_num_at_Q(dw3_dq1_normed, Q)
print(dw_dq_ref - dw_dq_python_normed[(3,1)])

# evaluate python implementation of dw_3 / dq_2
dw_dq_ref = eval_num_at_Q(dw3_dq2_normed, Q)
print(dw_dq_ref - dw_dq_python_normed[(3,2)])

# evaluate python implementation of dw_3 / dq_3
dw_dq_ref = eval_num_at_Q(dw3_dq3_normed, Q)
print(dw_dq_ref - dw_dq_python_normed[(3,3)])

[[-2.22044605e-16  0.00000000e+00 -1.11022302e-16 -5.55111512e-17]
 [ 1.11022302e-16  2.77555756e-17 -1.11022302e-16 -1.66533454e-16]
 [-5.89805982e-17 -8.32667268e-17 -1.11022302e-16  1.38777878e-16]
 [-6.93889390e-17 -5.55111512e-17  1.17961196e-16 -1.11022302e-16]]
[[ 0.00000000e+00 -5.55111512e-17  2.77555756e-17  0.00000000e+00]
 [-5.55111512e-17 -1.11022302e-16  8.32667268e-17 -5.55111512e-17]
 [ 4.16333634e-17  1.38777878e-16  0.00000000e+00 -2.22044605e-16]
 [ 5.55111512e-17  0.00000000e+00 -3.33066907e-16 -3.33066907e-16]]
[[-2.22044605e-16  1.52655666e-16 -5.55111512e-17  1.04083409e-17]
 [-2.22044605e-16  2.77555756e-16 -1.11022302e-16  2.77555756e-17]
 [ 1.11022302e-16 -5.55111512e-17  2.22044605e-16  1.11022302e-16]
 [ 6.24500451e-17  1.38777878e-17  8.32667268e-17  3.88578059e-16]]
[[ 2.77555756e-16 -1.38777878e-16  6.24500451e-17  3.81639165e-17]
 [-1.11022302e-16 -5.55111512e-17  2.35922393e-16  5.55111512e-17]
 [ 0.00000000e+00  1.94289029e-16  2.22044605e-16 -2.775557

This concludes the check. The recurrence relationships are valid and the Python implementation correctly implements these equations.